# Implementación y contraste de algoritmos de pronóstico de generación para una planta solar fotovoltaica de 80 kW de la Universidad de Los Andes

## Diego Andrés Acosta y Robinson Julián Portilla

#### En este proyecto de grado se realiza el pronóstico de generación del día siguiente de una planta solar fotovoltaica de 80 kW de la Universidad de Los Andes mediante diferentes algoritmos de Machine Learning. 

#### Se importan las librerías a utilizar 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from scipy import stats

## Pre-Procesamiento de Datos

En esta etapa se realiza el pre-procesamiento de los datos empleados para realizar los modelos de pronóstico.

Los datos se encuentran en la carpeta '/DATOS SOLAR' de este repositorio. Los datos tienen una resolución de 5 minutos y consisten en la fecha (dd/mm/aa), hora (hh:mm), Potencia generada (kW), irradiación solar (W/m^2), Temperatura de los modulos (°C), Temperatura ambiente (°C) e irradiación solar media (W/m^2). Se consideraron los datos de octubre de 2019 a diciembre de 2020 para realizar el entrenamiento y evaluación de los modelos implementados. 

Cabe señalar que, puesto que la estación meteorológica de la planta no otorga datos de temperatura ambiente, esta fue estimada mediante la siguiente expresión:

#### Tamb=Tcelda-G*(TONC-20°C)/800W/m^2

Donde: 
- G es la irradiación en ese momento en W/m^2
- TONC es la temperatura nominal de la celda en °C, la cual es de 42°C para los paneles LG 400W de la planta
- Tcelda es la temperatura de los modulos fotovoltaicos en °C en ese momento
- Tamb es la temperatura ambiente en °C 

Además, con el fin de mejorar el pronóstico de los modelos de series de tiempo, se suaviazan los datos de temperatura ambiente y de insolación solar mediante un promedio móvil de ventana 11. La columna Insolación solar media corresponde a los datos de Insolación solar suaviazados.  

### Lectura de datos:
En esta etapa se leen los datos almacenados en la carpeta '/DATOS SOLAR':

In [2]:
ruta = 'C:\\Users\\USUARIO\\OneDrive\\Universidad\\Proyecto de Grado\\Repositorio\\Proyecto-Electronica\\DATOS SOLAR\\'
ruta2='C:\\Users\Diego Acosta\\Documents\\GitHub\\Proyecto-Electronica\\DATOS SOLAR\\'
mes = input('Ingrese el mes: ')
año = input('Ingrese el año: ')
try:
    datos = pd.read_csv(ruta + mes + ' ' + año + '.csv')
except:
    datos = pd.read_csv(ruta2 + mes + ' ' + año + '.csv')
datos

Ingrese el mes: Octubre
Ingrese el año: 2020


,Fecha,Hora,Potencia [kW],Insolacion [W/m2],Temperatura de modulos [C],Temperatura Ambiente [C],Insolacion media [W/m2]
0,1/10/2020,0:00,x,0.98,11.05,11.448050,0.980000
1,1/10/2020,0:05,x,0.98,11.52,11.473050,0.980000
2,1/10/2020,0:10,x,0.98,11.62,11.490550,0.980000
3,1/10/2020,0:15,x,0.98,11.60,11.513050,0.980000
4,1/10/2020,0:20,x,0.98,11.55,11.555050,0.980000
...,...,...,...,...,...,...,...
8923,31/10/2020,23:35,x,0.97,13.05,13.436242,0.973000
8924,31/10/2020,23:40,x,0.97,12.96,13.369931,0.972222
8925,31/10/2020,23:45,x,0.97,13.16,13.297041,0.971250
8926,31/10/2020,23:50,x,0.97,13.48,13.249000,0.971429


### Funciones para el pre procesamiento: 

In [3]:
#Esta función reemplaza los datos ue tengan un valor de 'x' por un 0

def reemplazo(datos):
    datos.replace({'Potencia [kW]':'x'},0,inplace = True)
    datos.dropna(inplace = True)
    return datos

#Esta función toma los valores donde se observa que el sol está activo
# el intervalo de horas tomado es en un día de 6:00 am a 6:00pm. Además,
#en esta función se cambian algunos valores de temperatura ambiente 
#considerados fuera de lo normal para la ciudad de Bogotá

def intervalosolar(datos):
    list1 = datos['Hora'][0:72].tolist()
    list2 = datos['Hora'][217:288].tolist()
    
    list = list1+list2
    
    for i in range(len(list)):
        datos = datos[datos['Hora']!= list[i]]
    try:
        for i in datos['Temperatura ambiente [C]']:   
            if i<23.5:
                m=i
            if i >23.5:
                datos.replace({'Temperatura ambiente [C]':i},m,inplace=True)
    except:
        for i in datos['Temperatura Ambiente [C]']:   
            if i<23.5:
                m=i
            if i >23.5:
                datos.replace({'Temperatura Ambiente [C]':i},m,inplace=True)
    return datos



#En esta función se guarda el dataframe pre procesado en formato .csv
#en una carpeta nueva llamada '/Pre- procesados'
rutanueva = 'C:\\Users\\USUARIO\\OneDrive\\Universidad\\Proyecto de Grado\\Repositorio\\Proyecto-Electronica\\DATOS SOLAR'
def guardarcsv(datos,nombre):
    datos.to_csv(rutanueva+'\\Pre procesados\\'+ nombre)

#### Se crea una lista donde se guardan las rutas de los archivos a leer:

In [4]:
try:
    archivos = glob.glob('C:\\Users\\USUARIO\\OneDrive\\Universidad\\Proyecto de Grado\\Repositorio\\Proyecto-Electronica\\DATOS SOLAR\\*.csv')
except:
    archivos = glob.glob('C:\\Users\Diego Acosta\\Documents\\GitHub\\Proyecto-Electronica\\DATOS SOLAR\\*.csv')    

nombres = [] #Vector con los nombres de los archivos

for i in range(len(archivos)):
    nombre = archivos[i].split('\\')
    nombres.append(nombre[-1])
nombres

[]

### Ejecución de funciones:

#### Se lee cada uno de los archivos y se ejecutan las diferentes funciones definidas previamente:

In [5]:
for i in range(len(archivos)):
    #Se lee el archivo de la ruta
    datos = pd.read_csv(archivos[i])
    datos = reemplazo(datos)
    datos = intervalosolar(datos)
    guardarcsv(datos, nombres[i])


#### Se muestra un ejemplo de cómo queda el dataframe final:

In [6]:
datos

,Fecha,Hora,Potencia [kW],Insolacion [W/m2],Temperatura de modulos [C],Temperatura Ambiente [C],Insolacion media [W/m2]
0,1/10/2020,0:00,x,0.98,11.05,11.448050,0.980000
1,1/10/2020,0:05,x,0.98,11.52,11.473050,0.980000
2,1/10/2020,0:10,x,0.98,11.62,11.490550,0.980000
3,1/10/2020,0:15,x,0.98,11.60,11.513050,0.980000
4,1/10/2020,0:20,x,0.98,11.55,11.555050,0.980000
...,...,...,...,...,...,...,...
8923,31/10/2020,23:35,x,0.97,13.05,13.436242,0.973000
8924,31/10/2020,23:40,x,0.97,12.96,13.369931,0.972222
8925,31/10/2020,23:45,x,0.97,13.16,13.297041,0.971250
8926,31/10/2020,23:50,x,0.97,13.48,13.249000,0.971429


## Análisis estadístico y de desempeño de la planta solar fotovoltaica 

En esta sección se realiza el análisis estadístico y de desempeño de los datos de potencia generada, irradiación solar y temperatura ambiente registrados de octubre de 2019 a diciembre de 2020 por la estación meteorológica de la planta solar fotovoltaica. 

In [7]:
# Recibe un dataframe de datos de la planta y la variable a analizar

def soldiario(datos,variable = 'Insolacion [W/m2]'):
    fechas = datos['Fecha'].unique()
    solxdia = []
    for i in range(len(fechas)):
        solxdia.append(datos[datos['Fecha']==fechas[i]].sum()[variable]/12 )
    df = {'Dia':fechas, variable:solxdia}
    df = pd.DataFrame(df)
    return df

In [8]:
ruta = 'C:\\Users\\USUARIO\\OneDrive\\Universidad\\Proyecto de Grado\\Repositorio\\Proyecto-Electronica\\DATOS SOLAR\\Pre procesados\\'
ruta2 = 'C:\\Users\Diego Acosta\\Documents\\GitHub\\Proyecto-Electronica\\DATOS SOLAR\\Pre procesados\\'
try:
    archivospre = glob.glob(ruta +'*.csv')
except:
    archivospre = glob.glob(ruta2 +'*.csv')
archivospre

[]

In [9]:
nombres2 = []
for i in range(len(archivospre)):
    nombre = archivos[i].split('\\')
    nombres2.append(nombre[-1])
nombres2

[]

#### Se 

In [10]:
i=0
variable = 'Insolacion [W/m2]'
#variable = 'Potencia [kW]'

for nombre in nombres2:
    df = pd.read_csv(nombre)
    
    try: 
        df = df.drop('Unnamed: 0',axis=1)
    except:
        pass
    
    df = df.dropna()
    
    name = nombre.split('.') 
    name = name[0].split(' ')
    
    globals()[name[0] + name[1]] = soldiario(df,variable)
    
    if i == 0:
        todo = globals()[name[0] + name[1]] # DataFrame donde se guardan los diferentes valores de insolación por día.
    else:
        todo = pd.concat([todo , globals()[name[0] + name[1]]],axis = 0)
    i+=1

In [11]:
inso_max = todo[variable].max()
inso_min = todo[variable].min()

intervalos = np.linspace(inso_min, inso_max, 15)
print(intervalos)

NameError: name 'todo' is not defined

## Histograma de insolación media diaria

In [ ]:
plt.figure()
plt.hist(todo['Insolacion [W/m2]'],15,color = 'yellow',ec = 'k') #Histograma de frecuencias de insolación diaria media
#plt.hold(True)
# Graficando Gamma
a = 4.5 # parametro de forma.
gamma = stats.gamma(a)
x = np.linspace(gamma.ppf(0.01),
                gamma.ppf(0.99), 100)
fp = gamma.pdf(x) # Función de Probabilidad
plt.plot(x*todo['Insolacion [W/m2]'].max()/8, fp)
plt.show()

In [ ]:
plt.hist(todo['Insolacion [W/m2]'],15,color = 'yellow',ec = 'k')